This assignment uses uses the MNL model to analyze (1) yogurt purchase data made by consumers at a retail location, and (2) conjoint data about consumer preferences for minivans.


## 1. Estimating Yogurt Preferences

### Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 4 products, then either $y=3$ or $y=(0,0,1,0)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, size, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 4 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta} + e^{x_4'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=\delta_{i4}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 \times \mathbb{P}_i(4)^0 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$


### Yogurt Dataset

We will use the `yogurt_data` dataset, which provides anonymized consumer identifiers (`id`), a vector indicating the chosen product (`y1`:`y4`), a vector indicating if any products were "featured" in the store as a form of advertising (`f1`:`f4`), and the products' prices (`p1`:`p4`). For example, consumer 1 purchased yogurt 4 at a price of 0.079/oz and none of the yogurts were featured/advertised at the time of consumer 1's purchase.  Consumers 2 through 7 each bought yogurt 2, etc.

_todo: import the data, maybe show the first few rows, and describe the data a bit._


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

yogurt = pd.read_csv('/home/jovyan/Desktop/MGTA495-2/projects/Project 3/yogurt_data.csv')
yogurt.head()

In [ ]:
yogurt.shape

In [ ]:
yogurt[['y1','y2','y3','y4']].sum(axis = 0)

In [ ]:
yogurt.describe()

## Dataset Overview:
The dataset consists of several columns that capture various aspects of the yogurt purchasing decision. 
1- ID: A unique identifier for each customer
2- y1,y2,y3,y4: Binary indicators representing whether a customer preferred a particulat yogurt brand, 1 being preferred and 0 being not preferred
3- f1,f2,f3,f4: Binary indicators showing whether each yogurt was featrued in a promotional display
4- p1,p2,p3,p4: Prices of the respective yogurt brabds at the time of purchase

## Analysis Goals
Using the MNL Model, we aim to achieve the following:
1- ```Estimate Customer Preferences```: Determine which yogurt brand is most preferred based on estimated coefficients. 
2- ```Price Senstivity Analysis```: Understand how changes in prices influence consumer choices. 
3- ```Market Share Simulation```: Simulate market share changes under different pricing scenarios. 


Let the vector of product features include brand dummy variables for yogurts 1-3 (we'll omit a dummy for product 4 to avoid multi-collinearity), a dummy variable to indicate if a yogurt was featured, and a continuous variable for the yogurts' prices:  

$$ x_j' = [\mathbbm{1}(\text{Yogurt 1}), \mathbbm{1}(\text{Yogurt 2}), \mathbbm{1}(\text{Yogurt 3}), X_f, X_p] $$

$k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). 

What we would like to do is reorganize the data from a "wide" shape with $n$ rows and multiple columns for each covariate, to a "long" shape with $n \times J$ rows and a single column for each covariate.  As part of this re-organization, we'll add binary variables to indicate the first 3 products; the variables for featured and price are included in the dataset and simply need to be "pivoted" or "melted" from wide to long.  

Reshape and prep the data: The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate) 


In [ ]:
yogurt_long = pd.wide_to_long(yogurt, stubnames=['y','f', 'p'], i=['id'], j='product').reset_index()

# Add product brand dummies
yogurt_long['yogurt1'] = (yogurt_long['product'] == 1).astype(int)
yogurt_long['yogurt2'] = (yogurt_long['product'] == 2).astype(int)
yogurt_long['yogurt3'] = (yogurt_long['product'] == 3).astype(int)

# Filter for actual choices
#yogurt_long = yogurt_long[yogurt_long['y'] == 1]

# Rename columns and drop unnecessary ones
yogurt_long.rename(columns={'f': 'featured', 'p': 'price'}, inplace=True)

# Display the DataFrame
yogurt_features= yogurt_long[['yogurt1','yogurt2','yogurt3','featured','price']]
yogurt_long

### Estimation


In [ ]:
yogurt_labels = np.reshape(yogurt[['y1','y2','y3','y4']],(-1, ))
yogurt_labels.shape 

yogurt_id = yogurt['id'].repeat(4).reset_index(drop = True)


```The log-likelihood function```

In [ ]:
def multi_ll(beta, y, x, ids):
    
    x= x.to_numpy()
    prob = np.exp(x.dot(beta))
    df = pd.DataFrame({'prob':prob,'ids':ids})
    #sum = df.groupby('ids')['prob'].sum().repeat(4).reset_index(drop= True)
    sum_prob = df.groupby('ids')['prob'].transform('sum')
    probs = prob/sum_prob
    return -np.log(probs).sum()

In [ ]:
beta = np.ones(5)*0
y= yogurt_labels
x= yogurt_features
i= yogurt_id
x= x.to_numpy()
prob = np.exp(x.dot(beta))
df = pd.DataFrame({'prob':prob,'ids':i})
sum = df.groupby('ids')['prob'].transform('sum')
probs = prob/sum 
-np.log(probs).sum()


````


```{python}

#yogurt_features.shape
yogurt_labels.shape

Finding the MLEs for the 5 parameters ($\beta_1, \beta_2, \beta_3, \beta_f, \beta_p$). 


In [ ]:
# result = sp.optimize.minimize(multi_ll,np.ones(5)*0.1,(yogurt_labels, yogurt_features, yogurt_id),method = 'L-BFGS-B', options = {'gtol':1e-5, 'maxiter':500})
# print (result.x)

# Perform the optimization again
beta_initial = np.ones(5) * 0.1  # Ensure the initial beta values are reasonable
result = sp.optimize.minimize(multi_ll, beta_initial, (yogurt_labels, yogurt_features, yogurt_id), method='L-BFGS-B', options={'gtol': 1e-5, 'maxiter': 50})

### Discussion

Looking at the intercepts (0.09970444, 0.10002663 and -0.0056015) we can infer that yogurt 2 was the most preferred and yogurt 3 is the least preferred. 

Taking the diffnce between the beta of the most preferred and the least preferred ygurt we can estimate the price difference consumers are willing to pay for their preferred brand. 
As long as the most preferred yogurt was 12.75 cents/oz more expensive than the least preferred yogurt, consumers are willing to make that extra spend to go after thier preferred brand.  


In [ ]:
intercepts = result.x[:3]  
most_preferred = np.argmax(intercepts)
least_preferred = np.argmin(intercepts)

print("Intercepts for Yogurt 1, 2, and 3:", intercepts)
print("Most preferred yogurt is:", most_preferred + 1)
print("Least preferred yogurt is:", least_preferred + 1)

In [ ]:
#(0.10002663 +0.0056015) /0.00828118

price_coefficient = result.x[-1]  # Assuming the last coefficient is the price
dollar_benefit = (intercepts[most_preferred] - intercepts[least_preferred]) * -price_coefficient

print("Dollar benefit between the most and least preferred yogurt:", dollar_benefit)


One benefit of the MNL model is that we can simulate counterfactuals (eg, what if the price of yogurt 1 was $0.10/oz instead of $0.08/oz).

### Market Share Analysis of Yogurt Brands
In our analysis of consumer prefrences for yogurt brands, we utilized the Multinomial Logit (MNL) model to estimate the current market shares and simulate the impact of a price change on consumer choices. Following results were obtained: 
The estimated market shares indicated that 100% of the market share was attibuted to Yogurt 1:
- Yogurt 1: 100%
- Yogurt 2: 0%
- Yogurt 3: 0%
- Yogurt 4: 0%
```Impact of Price Increase on Yogurt 1```
To understand the sensitivity of consumer choices to price changes, we simulated a scenario where the price of Yogurt 1 was increased by $0.10 per unit. The new market shares after this price increase remained unchanged:

Yogurt 1: 100%
Yogurt 2: 0%
Yogurt 3: 0%
Yogurt 4: 0%
Key Insights
Uniform Preference: The current results indicate a uniform preference for Yogurt 1 among all consumers in the dataset. This outcome suggests that either all consumers chose Yogurt 1 or there is a potential issue with the dataset or model.

Price Insensitivity: Despite increasing the price of Yogurt 1 by $0.10, the market share for Yogurt 1 did not decrease. This implies that, within the context of the data and model, consumers are not sensitive to this price change.

Further Investigation Needed: The results are surprising and may indicate an underlying issue with the data or model. It is important to verify the dataset to ensure it accurately represents a diverse set of consumer preferences and to reassess the MNL model implementation to confirm it is correctly specified.

Conclusion
Our initial and revised analyses using the Multinomial Logit model both yielded unexpected results, with 100% of market share attributed to Yogurt 1 regardless of a price increase. This highlights the importance of data and model validation in ensuring accurate and meaningful insights. By addressing these potential issues, we can better understand consumer preferences and market dynamics, ultimately guiding more effective pricing and promotional strategies.


In [ ]:
def calculate_market_shares(beta, features):
    utilities = np.dot(features, beta)
    exp_utilities = np.exp(utilities)
    
    # Ensure exp_utilities is a 2D array for summing along axis 1
    if exp_utilities.ndim == 1:
        exp_utilities = exp_utilities.reshape(-1, 1)
    
    shares = exp_utilities / np.sum(exp_utilities, axis=1, keepdims=True)
    return shares.mean(axis=0)

# Current market shares
current_shares = calculate_market_shares(result.x, yogurt_features)
print("Current market shares:", current_shares)

# Simulate price increase for yogurt1
yogurt_features_adjusted = yogurt_features.copy()
yogurt_features_adjusted.loc[yogurt_features_adjusted['yogurt1'] == 1, 'price'] += 0.10

# New market shares after price increase
new_shares = calculate_market_shares(result.x, yogurt_features_adjusted)
print("New market shares after price increase to yogurt 1:", new_shares)

# Check if the market shares for yogurt 1 decrease
print("Do the yogurt 1 market shares decrease?", "Yes" if new_shares[0] < current_shares[0] else "No")


In [ ]:
print("Shape of beta:", beta.shape)
print("Shape of yogurt_features:", yogurt_features.shape)

## 2. Estimating Minivan Preferences


### Data

_todo: download the dataset from here:_ http://goo.gl/5xQObB 

_todo: describe the data a bit. How many respondents took the conjoint survey?  How many choice tasks did each respondent complete?  How many alternatives were presented on each choice task? For each alternative._


In [ ]:
data = pd.read_csv("/home/jovyan/Desktop/MGTA495-2/projects/Project 3/rintro-chapter13conjoint.csv")

In [ ]:
data.head()

In [ ]:
data['resp.id'].nunique()


In [ ]:
data['ques'].nunique()


In [ ]:
data['alt'].nunique()


In [ ]:
data.shape


Ans 15 sets of choices each set has 3 alternatives and 200 participants. each participant has 45 row data 1 for each choice alternative 

The attributes (levels) were number of seats (6,7,8), cargo space (2ft, 3ft), engine type (gas, hybrid, electric), and price (in thousands of dollars).

### Model

_todo: estimate a MNL model omitting the following levels to avoide multicollinearity (6 seats, 2ft cargo, and gas engine). Include price as a continuous variable. Show a table of coefficients and standard errors.  You may use your own likelihood function from above, or you may use a function from a package/library to perform the estimation._  


In [ ]:
from patsy import dmatrices
import statsmodels.api as sm

# Define the formula for the model
# Using 7 seats, 3ft cargo, and hybrid engine as base levels
formula = 'choice ~ C(seat, Treatment(6)) + C(cargo, Treatment("2ft")) + C(eng, Treatment("gas")) + price'

# Create design matrices
y, X = dmatrices(formula, data, return_type='dataframe')

# Fit the MNL model
mnl_model = sm.MNLogit(y, X)
mnl_result = mnl_model.fit()

# Display the summary of the model including coefficients and standard errors
mnl_result.summary()


### Results

_todo: Interpret the coefficients. Which features are more preferred?_
Compared to 8 seats, comsumers prefer 6 seats. in terms of cargo they prefer 3 ft over 2 ft of cargo. and gas engine over hybrid or electric  and they prefer low prices 
_todo: Use the price coefficient as a dollar-per-util conversion factor. What is the dollar value of 3ft of cargo space as compared to 2ft of cargo space?_


In [ ]:
# Extract the price coefficient
price_coef = mnl_result.params.loc['price'][0]

# Extract the coefficient for cargo space (3ft vs 2ft)
cargo_coef = mnl_result.params.loc['C(cargo, Treatment("2ft"))[T.3ft]'][0]

# Calculate the dollar value of 3ft of cargo space compared to 2ft
dollar_value_cargo = -cargo_coef / price_coef
dollar_value_cargo


willing to pay $2750 for 1ft extra cargo space 
_todo: assume the market consists of the following 6 minivans. Predict the market shares of each minivan in the market._

| Minivan | Seats | Cargo | Engine | Price |
|---------|-------|-------|--------|-------|
| A       | 7     | 2     | Hyb    | 30    |
| B       | 6     | 2     | Gas    | 30    |
| C       | 8     | 2     | Gas    | 30    |
| D       | 7     | 3     | Gas    | 40    |
| E       | 6     | 2     | Elec   | 40    |
| F       | 7     | 2     | Hyb    | 35    |


In [ ]:
new_data = pd.DataFrame({"minivan":['A','B','C','D','E','F'],
'Ones':[1,1,1,1,1,1],
'seats':['7','6','8','7','6','7'],
'cargo':['2','2','2','3','2','2'],
'engine':['Hyb','Gas','Gas','Gas', 'Elec','Hyb'],
'price':[30,30,30,40,40,35]})
new_data = pd.get_dummies(new_data,columns = ['seats','cargo','engine'])
new_data = new_data[['Ones','seats_7', 'seats_8', 'cargo_3', 'engine_Elec', 'engine_Hyb', 'price']].to_numpy()
probabilities = mnl_model.predict(new_data)
probabilities

In [ ]:
from numpy.linalg import cholesky
from scipy.stats import multivariate_normal

def predict_hier_mnl(model, data, nresp=1000):
    """
    Function to predict shares from a hierarchical multinomial logit model.
    - model: Should be a dictionary containing 'coef' and 'cov' from a fitted model.
    - data: DataFrame containing the set of designs for which you want to predict shares.
    - nresp: Number of respondents for simulation.
    """
    # Extract model information
    coef_mu = model['coef']
    coef_sigma = model['cov']
    
    # Generate draws from the multivariate normal distribution
    draws = multivariate_normal.rvs(mean=coef_mu, cov=coef_sigma, size=nresp)
    
    # Calculate utilities for each design and respondent
    utilities = np.dot(data, draws.T)
    
    # Calculate the exponent of utilities
    exp_utilities = np.exp(utilities)
    
    # Calculate shares by normalizing the exponentiated utilities
    shares = exp_utilities / exp_utilities.sum(axis=1, keepdims=True)
    
    # Calculate the average share across all respondents
    mean_shares = shares.mean(axis=0)
    
    # Combine mean shares with the original data
    result = data.copy()
    result['Predicted Share'] = mean_shares
    
    return result

# Example usage
# Define your model coefficients and covariance matrix
model = {
    'coef': np.array([...]),  # Replace [...] with your model coefficients
    'cov': np.array([...])    # Replace [...] with your model covariance matrix
}

# Prepare your data in the correct format as a DataFrame
data = pd.DataFrame([...])  # Replace [...] with your data

# Predict market shares
predicted_shares = predict_hier_mnl(model, data)


_hint: this example is taken from the "R 4 Marketing Research" book by Chapman and Feit. I believe the same example is present in the companion book titled "Python 4 Marketing Research".  I encourage you to attempt these questions on your own, but if you get stuck or would like to compare you results to "the answers," you may consult the Chapman and Feit books._
